# 正态性检验方法
**正态性检验（Normality Test）**是统计学中用于判断数据是否服从正态分布（高斯分布）的重要方法。正态分布是许多统计模型（如t检验、ANOVA、线性回归等）的基础假设，因此验证数据的正态性至关重要。

正态性检验方法主要有**Shapiro-Wilk检验**、Kolmogorov-Smirnov检验 (K-S检验)、Anderson-Darling检验

| 检验方法               | Shapiro-Wilk检验                          | Kolmogorov-Smirnov检验 (K-S检验)           | Anderson-Darling检验                     |
|------------------------|------------------------------------------|------------------------------------------|------------------------------------------|
| **适用场景**           | 小样本（n ≤ 50）效果更好                  | 大样本，或已知参考分布时                  | 中小样本，尤其关注尾部拟合               |
| **原假设 (H₀)**        | 数据来自正态分布                          | 数据与指定分布（如正态分布）一致           | 数据来自特定分布（如正态分布）            |
| **统计量特点**         | 基于样本数据的顺序统计量                  | 基于经验分布函数与理论分布的最大差异       | 加权平方差异（对尾部更敏感）              |
| **优点**               | 小样本功效高                              | 适用于任何分布比较，无需参数估计           | 对尾部异常敏感，适合极端值检测           |
| **缺点**               | 大样本时可能过于敏感（易拒绝）             | 对分布参数未知时功效较低（需Lilliefors修正） | 计算较复杂                               |
| **参数估计需求**       | 不需要预先估计分布参数                     | 若参数未知需估计（μ,σ），需用Lilliefors修正 | 通常需要估计分布参数                      |
| **适用数据类型**       | 连续数据                                  | 连续或离散数据                            | 连续数据                                 |
| **样本量限制**         | 3 ≤ n ≤ 5000（常见实现）                  | 无严格限制，但大样本更稳定                | 无严格限制                               |

# 数值法（统计检验法）

## Kolmogorov-Smirnov检验（K-S检验）

### 基本原理
**K-S检验**是一种非参数检验，通过比较样本经验分布函数（ECDF）与理论分布函数（或两个样本的ECDF）的最大垂直距离来判断分布是否一致。其核心思想是：
- 若样本来自目标分布，则ECDF与理论CDF（或另一样本的ECDF）应非常接近。
- 若两者差异过大，则拒绝原假设。

### 适用场景
- **已知分布参数的比较**（如标准正态分布 N(0,1)）
- **大样本数据**（n > 50）

### 零假设与备择假设
- 单样本检验：
    - 零假设（$H_0$）：样本来自指定的理论分布（如正态分布）。
    - 备择假设（$H_1$）：样本不来自该分布。
- 双样本检验：
    - 零假设（$H_0$）：两个样本来自同一分布。
    - 备择假设（$H_1$）：两个样本分布不同。

### 检验统计量
统计量$D$的计算公式为：
$$D = \sup_x |F_n(x) - F(x)|$$
- $F_n(x)$：样本的经验分布函数（ECDF）。
- $F(x)$：理论分布的累积分布函数（CDF）（单样本）或另一样本的ECDF（双样本）。
- $\sup_x$：对所有x值取上确界（最大垂直距离）。
- $D$值范围：[0, 1]，值越大，分布差异越显著。

### 检验步骤
#### 单样本K-S检验流程
- **数据排序**：将样本数据按升序排列。
- **计算ECDF**：
$$F_n(x_i) = \frac{i}{n}, \quad i = 1, 2, \ldots, n$$
- **计算理论CDF**：如正态分布 $F(x_i) = \Phi\left(\frac{x_i - \mu}{\sigma}\right)$。
- **求最大距离D**： 
$$D = \max \left( |F_n(x_i) - F(x_i)|, |F_n(x_{i-1}) - F(x_i)| \right)$$
- **计算p值**：根据D值查表或通过近似公式计算。
- **决策**：若$p < α$，拒绝H₀。

#### 双样本K-S检验流程
- 分别计算两个样本的ECDF $F_{n1}(x)$ 和 $F_{n2}(x)$。
- 计算最大距离D：$$D = \sup_x |F_{n1}(x) - F_{n2}(x)|$$
- 计算p值并判断。

### 单样本K-S检验实际案例

#### 案例背景
某电商平台在首页改版中测试了两种商品推荐算法：
- 旧版算法（A组）：基于协同过滤的推荐策略
- 新版算法（B组）：基于深度学习的推荐策略

实验运行一周后，需要验证B组用户的浏览时长分布是否服从正态分布（以便决定后续使用假设检验方法）。

In [7]:
import numpy as np
from scipy.stats import norm

# 假设B组有1000名用户，其浏览时长（秒）数据如下

# 设置随机种子保证可复现
np.random.seed(42)

# 生成B组用户浏览时长数据
browse_time = np.abs(norm.rvs(loc=60, scale=30, size=1000, random_state=0))  # 绝对值保证非负
browse_time = np.round(browse_time, 1)

print("B组浏览时长示例（前10条）:", browse_time[:10])
print(f"均值 = {np.mean(browse_time):.1f} 秒")
print(f"标准差 = {np.std(browse_time):.1f} 秒")

B组浏览时长示例（前10条）: [112.9  72.   89.4 127.2 116.   30.7  88.5  55.5  56.9  72.3]
均值 = 59.2 秒
标准差 = 28.5 秒


#### `stats.kstest()`
- 输入参数
   - `rvs`: 输入待检验的数据（一维或多维数组）
   - `cdf`: 指定理论分布
       - `norm`：正态分布，参数 loc=μ, scale=σ
       - `t`：t 分布，参数 df
       - `uniform`：均匀分布，参数 loc=a, scale=b−a
       - `expon`：指数分布，参数 loc=0, scale=1/λ
       - `chi2`：卡方分布，参数 df=自由度
       - `f`：F分布，参数 dfn=分子自由度，dfd=分母自由度
       - `lognorm`：对数正态分布，参数 s=σ, loc, scale=exp(μ)
       - `pareto`：帕累托分布，参数 b
       - `beta`：Beta 分布，参数 a, b
       - `poisson`：泊松分布，参数$\mu$=事件平均发生次数
   - `args`: 传递给理论分布cdf的参数（如正态分布的(mean, std)）
   - `alternative`: 备择假设的类型
       - `two-sided`：双尾检验（默认）
       - `less`：单尾检验（样本分布是否≤理论分布）
       - `greater`：单尾检验（样本分布是否≥理论分布）
   - `axis`: 对多维数组指定计算轴（如axis=0按列检验，axis=1按行检验）。默认值为None，会将数组展平为一维后检验
   - `nan_policy`: 如何处理 NaN 值：
        - `propagate`：结果为 NaN
        - `omit`：忽略 NaN
        - `raise`：抛出错误
- 返回值
    - `D`: K-S检验统计量（ECDF与理论CDF的最大距离，范围[0, 1]）
    - `p`: p值

#### `stats.kstest`代码实现

In [8]:
from scipy import stats
import matplotlib.pyplot as plt

# 单样本K-S检验
D, p = stats.kstest(
    browse_time, 
    'norm', 
    args=(np.mean(browse_time), np.std(browse_time, ddof=1))  # 使用样本均值和标准差
)

# 输出结果
print(f"K-S检验统计量 D = {D:.4f}")
print(f"p值 = {p:.4f}")

# 显著性判断
alpha = 0.05
if p > alpha:
    print(f"结论（α={alpha}）：不拒绝原假设，浏览时长服从正态分布")
else:
    print(f"结论（α={alpha}）：拒绝原假设，浏览时长不服从正态分布")

K-S检验统计量 D = 0.0268
p值 = 0.4626
结论（α=0.05）：不拒绝原假设，浏览时长服从正态分布


### 双样本K-S检验实际案例

#### 案例背景
某新闻资讯平台进行AB测试，比较新旧两种推荐算法对用户阅读时长的分布影响：
- A组（对照组）：1000名用户使用旧版算法（基于热门排序）
- B组（实验组）：1000名用户使用新版算法（基于个性化推荐）
- 分析目标：验证两组用户的阅读时长分布是否相同（若不同，需进一步分析差异点）。

In [5]:
import numpy as np
from scipy import stats
from scipy.stats import norm, pareto

# 生成模拟数据（单位：秒）
# A组：旧版算法，正态分布，μ=100, σ=15，样本数=1000
a_data = norm.rvs(loc=100, scale=15, size=1000, random_state=0)

# B组：新版算法，帕累托分布，shape=3.0，scale=50，样本数=1000
b_shape = 3.0
b_scale = 50
b_data = pareto.rvs(b_shape, loc=0, scale=b_scale, size=1000, random_state=0)

print(f"A组统计量: 均值={np.mean(a_data):.1f}, 标准差={np.std(a_data):.1f}")
print(f"B组统计量: 均值={np.mean(b_data):.1f}, 标准差={np.std(b_data):.1f}")

A组统计量: 均值=99.3, 标准差=14.8
B组统计量: 均值=75.5, 标准差=45.3


#### 确定假设
- 零假设（$H_0$）：两组阅读时长分布相同
- 备择假设（$H_1$）：两组分布不同

#### `stats.ks_2samp()`
- 输入参数
   - `data1`: 第一组样本数据
   - `data2`: 第二组样本数据
   - `alternative`: 备择假设的类型
       - `two-sided`：双尾检验（默认）：检验两个分布是否不同
       - `less`：检验 data1 分布是否小于 data2
       - `greater`：	检验 data1 分布是否大于 data2
   - `axis`: 沿哪个轴进行检验
   - `nan_policy`: 如何处理 NaN 值：
        - `propagate`：结果为 NaN
        - `omit`：忽略 NaN
        - `raise`：抛出错误
- 返回值
    - `statistic`: KS 检验统计量 D 值（CDF 最大差距）
    - `p`: p值

#### `stats.ks_2samp`代码实现

In [6]:
# 执行双样本K-S检验
D, p = stats.ks_2samp(a_data, b_data)

# 输出结果
print(f"\nK-S检验统计量 D = {D:.4f}")
print(f"p值 = {p:.4f}")

# 显著性判断
alpha = 0.05
if p > alpha:
    print(f"结论（α={alpha}）：无法拒绝H₀，两组分布可能相同")
else:
    print(f"结论（α={alpha}）：拒绝H₀，两组分布显著不同")


K-S检验统计量 D = 0.6780
p值 = 0.0000
结论（α=0.05）：拒绝H₀，两组分布显著不同
